# Download do Código Base

In [2]:
!gdown --id 1xEIku1VyGy6bh3FKEfdr-wtFVFpIsLAS

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1xEIku1VyGy6bh3FKEfdr-wtFVFpIsLAS
To: /content/rag_faq.tar.gz
100% 261k/261k [00:00<00:00, 4.59MB/s]


In [3]:
!tar -xvzf rag_faq.tar.gz

./
./config.yaml
./run_index.py
./ollama.serve.log
./server.log
./setup.py
./.ipynb_checkpoints/
./setup.cfg
./mistral-nemo.log
./projects/
./projects/rag_exemplo/
./projects/rag_exemplo/faq.csv
./projects/llm_test_project/
./projects/llm_test_project/faq_with_embeddings.csv
./projects/llm_test_project/embeddings.npy
./projects/llm_test_project/faq.csv
./prompts/
./prompts/rules.txt
./prompts/.ipynb_checkpoints/
./prompts/response.txt
./prompts/persona.txt
./rag_faq/
./rag_faq/server.py
./rag_faq/embedder.py
./rag_faq/utils.py
./rag_faq/.ipynb_checkpoints/
./rag_faq/retriever.py
./rag_faq/generator.py
./rag_faq/__pycache__/
./rag_faq/__pycache__/config.cpython-312.pyc
./rag_faq/__pycache__/embedder.cpython-312.pyc
./rag_faq/__pycache__/indexer.cpython-312.pyc
./rag_faq/__pycache__/indexer.cpython-311.pyc
./rag_faq/__pycache__/generator.cpython-311.pyc
./rag_faq/__pycache__/utils.cpython-312.pyc
./rag_faq/__pycache__/utils.cpython-311.pyc
./rag_faq/__pycache__/retriever.cpython-311.pyc


In [4]:
!pip install -e .

Obtaining file:///content
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Running setup.py develop for rag-faq
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [5]:
import pandas as pd
from rag_faq.config import load_config
from rag_faq.indexer import generate_faqs
from rag_faq.embedder import embed_faqs
import os
import time
import subprocess

# Instalando uma LLM

In [6]:
# 1. Instalar o Ollama
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [7]:
# 2. Iniciar o servidor Ollama em background

# Inicia o servidor
!nohup ollama serve > ollama.serve.log 2>&1 &

# Aguarda o servidor iniciar (IMPORTANTE!)
print("Aguardando servidor Ollama iniciar...")
time.sleep(5)

# Verifica se o servidor está respondendo
for i in range(10):
    try:
        result = subprocess.run(
            ["curl", "-s", "http://localhost:11434/api/tags"],
            capture_output=True,
            timeout=2
        )
        if result.returncode == 0:
            print("✅ Servidor Ollama está rodando!")
            break
    except:
        pass
    time.sleep(2)
    print(f"Tentativa {i+1}/10...")
else:
    print("❌ Servidor não respondeu. Verifique os logs:")
    !tail -20 ollama.serve.log

Aguardando servidor Ollama iniciar...
✅ Servidor Ollama está rodando!


In [8]:
# 3. Baixar o modelo
!ollama pull mistral-nemo

In [9]:
# 4. Testar o modelo
!ollama run mistral-nemo "Olá, mundo!"

Oi! Como posso ajudar você hoje?



# Configurando o Projeto de RAG

In [10]:
config = '''
llm:
  faq_generator:
    provider: http://localhost:11434/v1
    model: mistral-nemo
    temperature: 0.3
    api_key: TANTOFAZ

  rag_answer:
    provider: http://localhost:11434/v1
    model: mistral-nemo
    temperature: 0.3
    api_key: TANTOFAZ

embedding:
  model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

retrieval:
  top_k: 2

indexing:
  questions_per_text: 3

paths:
  projects_dir: ./projects
  prompts_dir: ./prompts
'''


path = "./config.yaml"
with open(path, "w", encoding="utf-8") as f:
    f.write(config)

In [11]:
# Load LLM-generated texts
df = pd.read_csv("data/PBL_3-data.csv")
texts = df["text"].tolist()

# Load config
config = load_config("config.yaml")

# Define project name
project_name = "pbl_3-star_wars"
project_dir = os.path.join(config["paths"]["projects_dir"], project_name)
os.makedirs(project_dir, exist_ok=True)

texts

['Em seu auge, o Império Galáctico se espalha por grande parte da galáxia conhecida de Star Wars, que consiste em milhões de sistemas estelares e bilhões de colônias, estaleiros, mundos-fortaleza e territórios externos. O Império substitui a República Galáctica no final das Guerras Clônicas orquestradas por Sheev Palpatine, que era então o Chanceler Supremo da República. Palpatine também é secretamente um Lorde das Trevas dos Sith chamado Darth Sidious, com a intenção de expurgar os Jedi e restaurar o Império Sith ao poder. O Império Galáctico nasceu do colapso da República Galáctica. No entanto, suas sementes são plantadas durante as Guerras Clônicas, a guerra épica entre a República e a Separatista Confederação de Sistemas Independentes. Por causa de um senado corrupto e de uma guerra civil, Palpatine foi facilmente capaz de assumir o controle da República.',
 'O Império Galáctico , também conhecido como Primeiro Império Galáctico ou simplesmente Império , e mais tarde como Antigo Im

Se tudo deu certo, foi criado um projetos/rag_exemplo

In [12]:
# Generate FAQs
generate_faqs(config, project_dir, texts)

Generating FAQs: 100%|██████████| 18/18 [02:39<00:00,  8.88s/it]

✅ FAQ saved to: ./projects/pbl_3-star_wars/faq.csv


In [13]:
# Generate embeddings
embed_faqs(config, project_dir)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating embeddings: 100%|██████████| 54/54 [00:01<00:00, 38.40it/s]

✅ Embeddings saved to: ./projects/pbl_3-star_wars


In [14]:
faq_path = os.path.join(project_dir, "faq.csv")

if os.path.exists(faq_path):
    try:
        df_faq = pd.read_csv(faq_path)
        print("Successfully read faq.csv:")
        display(df_faq)
    except pd.errors.EmptyDataError:
        print(f"Error: faq.csv at {faq_path} is empty.")
    except Exception as e:
        print(f"An error occurred while reading faq.csv: {e}")
else:
    print(f"Error: faq.csv not found at {faq_path}")

Successfully read faq.csv:


,source_text,question,answer
0,"Em seu auge, o Império Galáctico se espalha po...",O que causou o colapso da República Galáctica?,Um senado corrupto e uma guerra civil.
1,"Em seu auge, o Império Galáctico se espalha po...",Quem orquestrou as Guerras Clônicas?,"Sheev Palpatine, que era então o Chanceler Sup..."
2,"Em seu auge, o Império Galáctico se espalha po...",Qual era a intenção secreta de Palpatine como ...,Expurgar os Jedi e restaurar o Império Sith ao...
3,"O Império Galáctico , também conhecido como Pr...",Quem era o verdadeiro poder por trás do Império?,"Os Sith, liderados pelo Imperador Galáctico Da..."
4,"O Império Galáctico , também conhecido como Pr...",Como os Sith conseguiram tomar o controle da g...,Eles capitalizaram o sentimento anti-Jedi e an...
5,"O Império Galáctico , também conhecido como Pr...",Qual foi a primeira grande vitória dos rebelde...,A batalha de Mustafar em 4 ABY.
6,Os ideais imperiais do Império Galáctico podem...,O que causou o fim da República Galáctica como...,"A manipulação secreta de Darth Sidious, também..."
7,Os ideais imperiais do Império Galáctico podem...,Quem foi o último indivíduo a ocupar o cargo d...,"Darth Sidious, que se fazia passar publicament..."
8,Os ideais imperiais do Império Galáctico podem...,Como Darth Sidious conseguiu se tornar Supremo...,Ele orquestrou a Invasão de Naboo pela Federaç...
9,"Desde o início das Guerras Clônicas em 22 ABY,...",Quando começaram as Guerras Clônicas?,As Guerras Clônicas começaram em 22 ABY.


In [15]:
# Explorando RAG no terminal
# !rag-faq --mode query --project pbl_3-star_wars

# Opcionalmente, subir um servidor para explorar interface web.

In [16]:
# funciona localmente...
# !python -m rag_faq.server --project rag_exemplo --port 8089
